# Torch loss functions

> Loss functions that are implemented in PyTorch

In [ ]:
#| default_exp torch_utils.loss_functions

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

from typing import Union, Optional

import numpy as np
from ddopnew.utils import Parameter

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn.modules.loss import _Loss

import warnings

In [ ]:
# |export

def quantile_loss(
    input: torch.Tensor,
    target: torch.Tensor,
    quantile: torch.Tensor,
    reduction: str = 'mean',
) -> torch.Tensor:

    if not (target.size() == input.size()):
        warnings.warn(
            f"Using a target size ({target.size()}) that is different to the input size ({input.size()}). "
            "This will likely lead to incorrect results due to broadcasting. "
            "Please ensure they have the same size.",
            stacklevel=2,
        )

    expanded_input, expanded_target = torch.broadcast_tensors(input, target)

    # print(expanded_input.size(), expanded_target.size(), quantile.size())
    # print(quantile)

    loss = torch.max((expanded_target - expanded_input) * quantile, (expanded_input - expanded_target) * (1 - quantile))

    # print(losks.size())

    if reduction == 'mean':
        return loss.mean()
    elif reduction == 'sum':
        return loss.sum()
    else:
        raise ValueError(f"reduction={reduction} is not valid")

    return loss

In [ ]:
#| export

class TorchQuantileLoss(_Loss):

    """
    Implmentation of the quantile loss in Pytorch.
    Unlike the Numpy-based implementation ```quantile_loss``` in the loss_functions module, this implementation
    this implementation reduces the results to a scalar value using the specified reduction method. This class is 
    used to train Pytorch models using the quantile loss.
    """

    __constants__ = ['reduction']
    def __init__(self,reduction: str = 'mean') -> None: #
        super().__init__(reduction=reduction)

    def forward(self, input: torch.Tensor, target: torch.Tensor, quantile: Parameter | np.ndarray) -> torch.Tensor: #

        """
        Forward pass of the quantile loss function.

        """

        quantile = self.convert_quantile(quantile, input_dtype=input.dtype, device=input.device)
            
        if not (target.shape == input.shape == quantile.shape):
            warnings.warn(
                f"Mismatch in dimensions: target dimension ({target.shape}), input dimension ({input.shape}), "
                f"and quantile dimension ({quantile.shape}) must be the same. "
                "This will likely lead to incorrect results due to broadcasting. "
                "Please ensure they have the same size.",
                stacklevel=2,
            )

        return quantile_loss(input, target, quantile, reduction=self.reduction)
    
    def convert_quantile(self, quantile: Parameter | np.ndarray, input_dtype: torch.dtype = torch.float32, device: torch.device = torch.device('cpu')) -> torch.Tensor:
        
        if isinstance(quantile, Parameter):
            quantile =  quantile.get_value()
        elif isinstance(quantile, np.ndarray):
            quantile = torch.tensor(quantile, dtype=input_dtype, device=device)
        elif isinstance(quantile, torch.Tensor):
            # ensure dtype and device are the same as the input tensor
            quantile = quantile.to(dtype=input_dtype, device=device)
        else:
            raise ValueError(f"quantile must be of type Parameter, np.ndarray, or torch.Tensor, but got {type(quantile)}")

        return quantile


In [ ]:
show_doc(TorchQuantileLoss, title_level=2)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/torch_utils/loss_functions.py#L53){target="_blank" style="float:right; font-size:smaller"}

## TorchQuantileLoss

>      TorchQuantileLoss (reduction:str='mean')

*Implmentation of the quantile loss in Pytorch.
Unlike the Numpy-based implementation ```quantile_loss``` in the loss_functions module, this implementation
this implementation reduces the results to a scalar value using the specified reduction method. This class is 
used to train Pytorch models using the quantile loss.*

|    | **Type** | **Default** | **Details** |
| -- | -------- | ----------- | ----------- |
| reduction | str | mean |  |
| **Returns** | **None** |  |  |

In [ ]:
show_doc(TorchQuantileLoss.forward)

---

[source](https://github.com/opimwue/ddopnew/blob/main/ddopnew/torch_utils/loss_functions.py#L66){target="_blank" style="float:right; font-size:smaller"}

### TorchQuantileLoss.forward

>      TorchQuantileLoss.forward (input:torch.Tensor, target:torch.Tensor,
>                                 quantile:ddopnew.utils.Parameter|numpy.ndarray
>                                 )

*Forward pass of the quantile loss function.*

|    | **Type** | **Details** |
| -- | -------- | ----------- |
| input | Tensor |  |
| target | Tensor |  |
| quantile | ddopnew.utils.Parameter \| numpy.ndarray |  |
| **Returns** | **Tensor** |  |

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()